In [1]:
# # IMPORTS
# astronomy
from astropy.io import fits
# plotting 
from matplotlib.ticker import AutoMinorLocator
import matplotlib.pyplot as plt
# data 
import numpy as np
# my python files 
import match
import v2_AGN_DataAndPlotting as adp
import v2_SED as SED

# Prep Data

In [29]:
def PrintNumber(myDict) :
    for key,val in myDict.items():
        if(isinstance(val,np.ndarray)):
            print(key,'\t',len(val)) 
        elif(isinstance(val,dict)):
            print(key,end='')
            for key1,val1 in val.items():
                if(isinstance(val1,dict)) : 
                    print('\t',key1,end='')
                    for key2,val2 in val1.items():
                        print('\t\t',key2,'\t',len(val2))
                else : 
                    print('\t',key1,'\t',len(val1))

In [3]:
# read AGN IDs
agnIDs = adp.ReadFile(adp.path_csv+'\\v2_AGNid_tricolor.csv')

# convert agnIDs into dict of integer numpy arrays 
agnIDs_dict = {
    'RED' : np.array(agnIDs[0], dtype=int),
    'GRN' : np.array(agnIDs[1], dtype=int),
    'BLU' : np.array(agnIDs[2], dtype=int)
}

# verify nmber of sources
PrintNumber(agnIDs_dict)

Cropped:  ['Row 1 = MIR exclusive AGN with id, redshift, and IRAC (n=472). Row 2 = MIR-X-ray inclusive AGN with id, redshift, and IRAC (n=663). Row 3 = X-ray exclusive AGN with id, redshift, and IRAC (n=1717). ']
RED 	 472
GRN 	 663
BLU 	 1717


In [4]:
# get redshifts
zDict = adp.ReadRedshifts(file=adp.path_cat+'\COSMOS_z_matches.csv')

Cropped:  ['ID', 'z']
Number of redshifts: 485793


In [5]:
agnInfo = {}
for key,val in agnIDs_dict.items():
    # match redshift to AGN
    key_zid, key_idz = match.match(zDict['ID'], val)
    agnInfo[key] = {
        'ID' : val[key_idz],
        'z'  : zDict['z'][key_zid]
    }
# verify number of sources
PrintNumber(agnInfo)

RED	 ID 	 472
	 z 	 472
GRN	 ID 	 663
	 z 	 663
BLU	 ID 	 1717
	 z 	 1717


In [6]:
# open COSMOS 2020 catalog file and get data 
with fits.open(adp.path_cat+'\\COSMOS2020_CLASSIC_R1_v2.0_master.fits') as hdul20:
    data20 = hdul20[1].data

In [7]:
# open COSMOS 2016 catalog file and get data 
with fits.open(adp.path_cat+'\\chandra_COSMOS_legacy_opt_NIR_counterparts_20160113_4d.fits') as hdul16:
    data16 = hdul16[1].data
    
# get data arrays from columns
id_data16 = np.array(data16['id_k_uv'])
print('.', end='')
L0510_data16 = np.array(data16['Lx_0510']) # x-ray luminosity in the 0.5-10 KeV band
print('.', end='')
k0510_data16 = np.array(data16['abs_corr_0510'])  # absorbtion correction coefficient in the 0.5-10 KeV band
print('.', end='')

...

In [8]:
# get starting info 
Fnu_uJy = SED.GetPhotometry(data20, print=False)
IDs_all = SED.GetID(data20, print=False)
lam_A   = SED.GetObservedWavelengths_A(print=False)

In [9]:
# interpolates the luminosity at 1um
def Lum_at6um(lamFlam,lam,z) :
    lum_list = []
    for x,y,z in zip(lam,lamFlam,z) : 
        # interpolate
        f = SED.Interpolate_log(x,y)
        # normalize at 1um
        Fnu_at1um = SED.Flog_X(f,6*1E+4) # 6A * 10^4 = 6um
        # convert to luminosity
        lum_list.append(SED.Flux_to_Lum(Fnu_at1um,z))
    return(np.array(lum_list))

In [10]:
for key,val in agnInfo.items():
    # match photometry to AGN
    key_agn, key_phot = match.match(val['ID'], IDs_all)
    # get phot
    Fnu = Fnu_uJy[key_phot]
    # get rest wavelength 
    lamR = SED.ConvertToRestWavelength(val['z'], print=False)
    # get energy density and normalize
    lamFlam = SED.ConvertToEnergyDensity(lamR, Fnu, print=False)
    # get luminosity at 1um
    lum6um = SED.Lum_at1um(lamFlam, lamR, val['z'], )
    # append info
    val['Rest Wavelength'] = lamR
    val['Energy Density'] = lamFlam
    val['Luminosity at 6um'] = lum6um

# verify number of sources
PrintNumber(agnInfo)

RED	 ID 	 472
	 z 	 472
	 Rest Wavelength 	 472
	 Energy Density 	 472
	 Luminosity at 6um 	 472
GRN	 ID 	 663
	 z 	 663
	 Rest Wavelength 	 663
	 Energy Density 	 663
	 Luminosity at 6um 	 663
BLU	 ID 	 1717
	 z 	 1717
	 Rest Wavelength 	 1717
	 Energy Density 	 1717
	 Luminosity at 6um 	 1717


In [11]:
agnInfo_yX = {}
agnInfo_nX = {}
for colorKey,agnValDict in agnInfo.items():
    ### get AGN with a good Lx
    # get key to the IDs from C-COSMOS data, which are parallel to Xray data
    key_agnID, key_id = match.match(agnValDict['ID'], id_data16)
    # get xray luminosity data
    Lx_agn = adp.IntrinsicLuminosity(L0510_data16[key_id], k0510_data16[key_id])
    mask_Lx_agn_good = Lx_agn > 0
    # build 'yes Xray' dict 
    agnDict_yX = {}
    for dictKey, dictVal in agnValDict.items() : 
        agnDict_yX[dictKey] = dictVal[(key_agnID[mask_Lx_agn_good])]
    agnDict_yX['Lx'] = Lx_agn[mask_Lx_agn_good]
    ### get remaining AGN
    # match all agn to those with Lx
    key_fullLx, key_Lx = match.match(agnValDict['ID'], agnDict_yX['ID'])
    # get AGN without Lx
    mask_noLx = np.ones(agnValDict['ID'].shape, dtype=bool)
    mask_noLx[key_fullLx] = False 
    # build 'no Xray' dict 
    agnDict_nX = {}
    for dictKey, dictVal in agnValDict.items() : 
        agnDict_nX[dictKey] = dictVal[(mask_noLx)]
    ### finish
    agnInfo_yX[colorKey] = agnDict_yX
    agnInfo_nX[colorKey] = agnDict_nX

# verify number of sources
print('~~~~ yX ~~~~~')
PrintNumber(agnInfo_yX)
print('~~~~ nX ~~~~~')
PrintNumber(agnInfo_nX) # only RED should have nX

~~~~ yX ~~~~~
RED	 ID 	 25
	 z 	 25
	 Rest Wavelength 	 25
	 Energy Density 	 25
	 Luminosity at 6um 	 25
	 Lx 	 25
GRN	 ID 	 663
	 z 	 663
	 Rest Wavelength 	 663
	 Energy Density 	 663
	 Luminosity at 6um 	 663
	 Lx 	 663
BLU	 ID 	 1717
	 z 	 1717
	 Rest Wavelength 	 1717
	 Energy Density 	 1717
	 Luminosity at 6um 	 1717
	 Lx 	 1717
~~~~ nX ~~~~~
RED	 ID 	 447
	 z 	 447
	 Rest Wavelength 	 447
	 Energy Density 	 447
	 Luminosity at 6um 	 447
GRN	 ID 	 0
	 z 	 0
	 Rest Wavelength 	 0
	 Energy Density 	 0
	 Luminosity at 6um 	 0
BLU	 ID 	 0
	 z 	 0
	 Rest Wavelength 	 0
	 Energy Density 	 0
	 Luminosity at 6um 	 0


In [19]:
# returns array of ( min <= z < max )
def GetMaskFromRedshiftsInRange(
        zArr,    # np array of z  
        min=0,   # minimum
        max=99,  # maximum 
        incusiveMin=False,
) : 
    # get mask of indecies that are in redshift range (true)
    if(incusiveMin):
        return (zArr >= min) & (zArr < max)
    else: 
        return (zArr >  min) & (zArr <= max)

In [30]:
def GetAGNinZrange(agnInfo_Xx : dict[str,dict[str,np.array]]):
    agnInfo_xX_byZ = {}
    for colorKey,agnDict in agnInfo_Xx.items() : 
        agnInfo_xX_byZ[colorKey] = {}
        # get masks for each redshift bin 
        zArray = agnDict['z']
        zMasks = {
            '$0 < z \leq 1$'   : GetMaskFromRedshiftsInRange(zArray,          max=1  ),
            '$1 < z \leq 1.5$' : GetMaskFromRedshiftsInRange(zArray, min=1,   max=1.5),
            '$1.5 < z \leq 2$' : GetMaskFromRedshiftsInRange(zArray, min=1.5, max=2  ),
            '$2 < z \leq 3$'   : GetMaskFromRedshiftsInRange(zArray, min=2,   max=3  ),
            '$3 < z \leq 6$'   : GetMaskFromRedshiftsInRange(zArray, min=3           )
        }
        for zKey, zArr in zMasks.items() : 
            agnInfo_xX_byZ[colorKey][zKey] = {}
            for agnDictKey, agnDictVal in agnDict.items() : 
                agnInfo_xX_byZ[colorKey][zKey][agnDictKey] = agnDictVal[zArr]
    return agnInfo_xX_byZ

agnInfo_yX_byZ = GetAGNinZrange(agnInfo_yX)
agnInfo_nX_byZ = GetAGNinZrange(agnInfo_nX)

# verify number of sources
print('~~~~ yX ~~~~~')
PrintNumber(agnInfo_yX_byZ)
print('~~~~ nX ~~~~~')
PrintNumber(agnInfo_nX_byZ) # only RED should have nX

~~~~ yX ~~~~~
RED	 $0 < z \leq 1$		 ID 	 25
		 z 	 25
		 Rest Wavelength 	 25
		 Energy Density 	 25
		 Luminosity at 6um 	 25
		 Lx 	 25
	 $1 < z \leq 1.5$		 ID 	 0
		 z 	 0
		 Rest Wavelength 	 0
		 Energy Density 	 0
		 Luminosity at 6um 	 0
		 Lx 	 0
	 $1.5 < z \leq 2$		 ID 	 0
		 z 	 0
		 Rest Wavelength 	 0
		 Energy Density 	 0
		 Luminosity at 6um 	 0
		 Lx 	 0
	 $2 < z \leq 3$		 ID 	 0
		 z 	 0
		 Rest Wavelength 	 0
		 Energy Density 	 0
		 Luminosity at 6um 	 0
		 Lx 	 0
	 $3 < z \leq 6$		 ID 	 0
		 z 	 0
		 Rest Wavelength 	 0
		 Energy Density 	 0
		 Luminosity at 6um 	 0
		 Lx 	 0
GRN	 $0 < z \leq 1$		 ID 	 100
		 z 	 100
		 Rest Wavelength 	 100
		 Energy Density 	 100
		 Luminosity at 6um 	 100
		 Lx 	 100
	 $1 < z \leq 1.5$		 ID 	 151
		 z 	 151
		 Rest Wavelength 	 151
		 Energy Density 	 151
		 Luminosity at 6um 	 151
		 Lx 	 151
	 $1.5 < z \leq 2$		 ID 	 161
		 z 	 161
		 Rest Wavelength 	 161
		 Energy Density 	 161
		 Luminosity at 6um 	 161
		 Lx 	 161
	 $2 < z \